In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

import re
import numpy as np
from keras.models import model_from_json
import gdown

#path='/content/drive/My Drive/Colab Notebooks/'

# загрузка модели
url = 'https://raw.githubusercontent.com/IlyaSk10/stress_symbol_predict/master/model_3.json'
output = "model_3.json"
gdown.download(url, output, quiet=False)
json_file = open("model_3.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

# загрузка весов
url = 'https://github.com/IlyaSk10/stress_symbol_predict/raw/master/model_3.h5'
output = "model_3.h5"
gdown.download(url, output, quiet=False)
model.load_weights("model_3.h5")
print("Loaded model from disk")

MAXLEN=40
CHARS = ['-', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к',
         'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш',
         'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё']
CHAR_INDICES = dict((c, i) for i, c in enumerate(CHARS))

# функция преобразования слова в вектор
def input_word(word):
    x = np.zeros((1, MAXLEN, len(CHAR_INDICES)))
    for index, letter in enumerate(word.rstrip()):
        x[0, index, CHAR_INDICES[letter]] = 1
    return x


# функция расставления знаков ударения на слова
def word_stress(sentence,stress_symbol="'"):
    tokens=re.findall(r'[а-яА-ЯёЁ]+',sentence)
    tokens=[i for i in tokens if len(i)>1]
    #print(tokens)
    for token in tokens:
        C=input_word(token.lower()).reshape(1,MAXLEN,len(CHARS))
        position=int(np.where(model.predict(C)==model.predict(C).max())[1])
        #print(position)
        repl=token[:position]+stress_symbol+token[position:]
        #print(repl)
        #repl=token[:position]+stress_symbol+token[position:]
        sentence=re.sub(token,repl,sentence)
    return sentence

  

In [6]:
sentence='В 1766 году, в ходе археологических раскопок в Олимпии, были обнаружены спортивные и храмовые сооружения. В 1875 году археологические исследования и раскопки продолжились под немецким руководством.'
word_stress(sentence,stress_symbol="'")

"В 1766 го'ду, в хо'де археологи'ческих раскопо'к в Оли'мпии, бы'ли обнару'жены спорти'вные и хра'мовые сооруже'ния. В 1875 го'ду археологи'ческие иссле'дования и раско'пки продолжи'лись по'д неме'цким руково'дством."